In [1]:
import os, rcmd, sys, plot_syn
from IPython.core.display import Image, display
%matplotlib inline

# Installation of the tool from "pristine" directory and running it for an experiment  with synthetic data ("identical twin")


In [ ]:
rcmd.run_command('./createdir_example_synthetic.sh')

In [ ]:
cd tool_intro/example_synthetic/

In [ ]:
ls -ralt

### Terminate installation by setting up link to Signature Simulator installed on system:
(This has to be done only once)

In [ ]:
# terminate the installation
rcmd.run_command('make setup')

In [ ]:
ls -ralt input/

### The protoype retrieval tool is shipped with a Makefile providing dedicated targets that illustrate it's basic usage
We will be looking into a target which runs a retrieval with synthetic data for both S1 and S2:

In [5]:
cat ../mk.complete.txt

complete: simulate.x runretrieval.x
	$(MAKE) clean_exp
	$(MAKE) presynthetic
	$(MAKE) synthetic
	$(MAKE) retrieval
	$(MAKE) save_exp SAVDIR=resdir_$(EXPV)_$@


### First we need to compile the drivers
The drivers `simulate.x` (to create synthetic observations) and `runretrieval.x` (which performs the actual retrieval including target processing) are implemented in Fortran.

In [ ]:
rcmd.run_command('make simulate.x runretrieval.x')

### Preproccessing step
It's task is to prepare the required input for creation of synthetic observations and performing the actual retrieval The preprocessor is a python script and is interfacing with the Fortran drivers via a set of Fortran-namelist and NetCDF files.

In [ ]:
cat ../mk.pre.txt

In [ ]:
rcmd.run_command('bin/rs_pre.py pre_synthetic -h')

Extra target retrieval times of interest (without presence of an observation) can be specified by a so called
`target schedule file`:

In [ ]:
cat input/target_schedule_2017.txt

Run preprocessing (here for the period May 2017)...

In [ ]:
rcmd.run_command('make presynthetic MISSION_LST="S1A S1B S2A S2B"')

In [ ]:
ls -ralt

We take a short look into two of the generated files only.
`site.nml` containes site-specific parameters that control the simulation (for the synthetic case settings suitable for the Munich site are applied). `retrctl.nml` is dedicated to control the inversion.

In [ ]:
cat site.nml

The initial guess for the control vector when running the inversion procedure, will be the prior perturbed by 25%!

In [ ]:
cat retrctl.nml

![Cost function components](cost_fn.png)

In [ ]:
cat ../mk.complete.txt

### Now start creation of synthetic observations...
This means here, we are generating observations with the `prior control vector`!

In [ ]:
rcmd.run_command('make synthetic -n')

In [ ]:
rcmd.run_command('make synthetic')

In [ ]:
ls -ralt

### ... that is how they look like:

In [ ]:
plot_syn.plot_obs_s1('obs_s1.nc')

In [ ]:
plot_syn.plot_obs_s2('obs_s2.nc')

### Based on this "observational data" and the settings created by the preprocessor, we can now start with the retrieval

In [ ]:
rcmd.run_command('make retrieval -n')

In [ ]:
rcmd.run_command('./runretrieval.x --help')

Here, to speed up things, we do this step without retrieving `FAPAR` and `HH backscatter` (in particular retrieving can be expensive in time)

In [ ]:
rcmd.run_command('make retrieval RETRARGXTRA=--no_targets')

Save set of involved files for this experiment in an extra directory (optional, of course)

In [ ]:
rcmd.run_command('make save_exp SAVDIR=ex_complete')

In [ ]:
ls -ralt ex_complete

In [ ]:
plot_syn.plot_iterate('ex_complete/iterate.dat', var='fct')

In [ ]:
plot_syn.plot_iterate('ex_complete/iterate.dat', var='grad')

### Since we were running an identical twin experiment, the retrieval procedure should be capable of recovering the "true" control vector

In [ ]:
plot_syn.plot_state('lai', _dir='ex_complete')